# Simulation

In [1]:
import numpy as np
from rob2004.interface import RobotInterfaceSim

In [2]:
robot = RobotInterfaceSim()

In [ ]:
state = robot.getJointState()

In [ ]:
robot.addMarker([0,0,0.5])

In [ ]:
robot.robot.getJointState()

In [1]:
from rob2004.sim import MujocoRobot
robot = MujocoRobot(fixed_body=True, visualizer_mode=True)

In [4]:
robot.add_visual_ball([0.0, 0.0, 0.13])

# MuJoCo XML Parsing

In [ ]:
from rob2004.rerun import RerunVisualizer
vis = visualizer = RerunVisualizer(app_name='logger', spawn=True)

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
info = ET.parse('/home/mim-server/projects/rooholla/NYU_ROB_UY_2004/rob2004/assets/robot/mujoco/pupper_v3_complete.position.fixed_base.xml')
root = info.getroot()
base = root.find('worldbody/body')
chains = base.findall('body')

In [ ]:
from scipy.spatial.transform import Rotation as R
def attrib2Pose(attrib):
    if 'pos' in attrib:
        link_pos = [float(n) for n in attrib['pos'].split(' ')] 
    else:
        link_pos = [0, 0, 0]
    if 'quat' in attrib:
        link_quat = [float(n) for n in attrib['quat'].split(' ')] 
    else:
        link_quat = [1, 0, 0 , 0]
    link_quat = link_quat[1:]+[link_quat[0]]
    link_R = R.from_quat(link_quat).as_matrix()
    T = np.eye(4)
    T[:3, :3] = link_R
    T[:3, -1] = link_pos
    return T

def rq2Pose(r, q):
    link_quat = q
    link_quat = link_quat[1:]+[link_quat[0]]
    link_R = R.from_quat(link_quat).as_matrix()
    T = np.eye(4)
    T[:3, :3] = link_R
    T[:3, -1] = r
    return T

In [ ]:
kinematic_at_rest = {}
for n in range(4):
    attrib = base.findall('body')[n].attrib
    name = attrib['name']
    B0_T_B1 = attrib2Pose(attrib)
    attrib = base.findall('body')[n].findall('body')[0].attrib
    B1_T_B2 = attrib2Pose(attrib)
    attrib = base.findall('body')[n].findall('body')[0].findall('body')[0].attrib
    B2_T_B3 = attrib2Pose(attrib)
    
    attrib = base.findall('body')[n].findall('body')[0].findall('body')[0].findall('site')[0].attrib
    B3_T_Foot = attrib2Pose(attrib)

    kinematic_at_rest[name] = dict(
        B0_T_B1=B0_T_B1,
        B1_T_B2=B1_T_B2,
        B2_T_B3 = B2_T_B3,
        B3_T_Foot = B3_T_Foot,
    )

In [ ]:
def forwardKinematics(chain):
    B0_T_B1 = chain['B0_T_B1']
    B1_T_B2 = chain['B1_T_B2']
    B2_T_B3 = chain['B2_T_B3']
    B0_T_B2 = B0_T_B1@B1_T_B2
    B0_T_B3 = B0_T_B2@B2_T_B3
    B0_T_Foot = B0_T_B3@chain['B3_T_Foot']
    return dict(
        B0_T_B1 = B0_T_B1,
        B0_T_B2 = B0_T_B2, 
        B0_T_B3 = B0_T_B3,
        B0_T_Foot = B0_T_Foot
    )    

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
T_offset = np.eye(4)
rot = R.from_euler('x', -90, degrees=True).as_matrix()
rot2 = R.from_euler('z', 90, degrees=True).as_matrix()
T_offset[:3, :3] = rot2@rot

vis.logMeshFile("/home/mim-server/projects/rooholla/NYU_ROB_UY_2004/rob2004/assets/robot/meshes/dae/Body V4 v70.001.dae",
                T_offset, 'robot/base', alpha=0)

fk = forwardKinematics(kinematic_at_rest['leg_front_r_1'])
B0_T_B1 = fk['B0_T_B1']
B1_T_mesh = rq2Pose([0, 0, -0.028], [1.32679e-06, 0, 0, 1])
B0_T_mesh = B0_T_B1@B1_T_mesh
vis.logMeshFile("/home/mim-server/projects/rooholla/NYU_ROB_UY_2004/rob2004/assets/robot/meshes/dae/Leg Assembly For Flanged v26.010.dae",
                B0_T_mesh, 'robot/leg_front_l_1')

B0_T_B2 = fk['B0_T_B2']
B2_T_mesh = rq2Pose([-0.028, 0, 0],[-2.93388e-06, 0.707108, 2.26082e-06, 0.707106])
B0_T_mesh = B0_T_B2@B2_T_mesh
vis.logMeshFile("/home/mim-server/projects/rooholla/NYU_ROB_UY_2004/rob2004/assets/robot/meshes/dae/Leg Assembly For Flanged v26.006.dae",
                B0_T_mesh, 'robot/leg_front_l_2')

B0_T_B3 = fk['B0_T_B3']
B2_T_mesh = rq2Pose([0.0685, 0.0494, -0.028],[1.32679e-06, 0, 0, 1])
B0_T_mesh = B0_T_B3@B2_T_mesh
vis.logMeshFile("/home/mim-server/projects/rooholla/NYU_ROB_UY_2004/rob2004/assets/robot/meshes/dae/Leg Assembly For Flanged v26.007.dae",
                B0_T_mesh, 'robot/leg_front_l_3')


import time
vis.logCoordinateFrame(np.eye(4), f'world/0', axis_length=0.05)

for name in kinematic_at_rest:
    if name=='leg_front_r_1':
        chain = kinematic_at_rest[name]
        result = forwardKinematics(chain)
        for k, T in enumerate(result.values()):
            time.sleep(0.5)
            vis.logCoordinateFrame(T, f'world/{name}/{k}', axis_length=0.05)
    
            
time.sleep(0.5)
vis.logCoordinateFrame(T, f'world/{name}/{k}', axis_length=0.05)